### Generate Credentials 

This notebook generates EC2 Credentials for mounting a public object store container from a Chameleon CLoud site. 

In [ ]:
#Import required packages
from openstack import connection
from chi import context
import swiftclient
import chi, os

In [ ]:
context.choose_project() 
context.choose_site(default="CHI@UC") # replace with your site 

## Authentication for Object Store

This step is used to **authenticate JupyterHub with the object store**.  
Since our JupyterHub session already has the necessary Chameleon credentials,
we don’t type in usernames or passwords here.  

The code below simply asks JupyterHub for a valid **token** and the **object store
endpoint**, and then uses them to connect with our object store services to create the buckets

## Establish connection with openstack

In [ ]:
# Authenticate - getting object store endpoint
os_conn = chi.clients.connection()
conn = connection.from_config()
token = os_conn.authorize()
storage_url = os_conn.object_store.get_endpoint()
# Connect to Swift (S3-compatible) object store
swift_conn = swiftclient.Connection(preauthurl=storage_url,
                                    preauthtoken=token,
                                    retries=5)

### Create EC2 credentials

the following script will generate that for us or you can head to create it using the GUI: 


In [ ]:
conn = connection.from_config()

project_id = conn.current_project_id 
identity_ep = conn.session.get_endpoint(service_type='identity', interface='public')
url = f"{identity_ep}/v3/users/{conn.current_user_id}/credentials/OS-EC2"
ID = {"tenant_id": project_id}

resp = conn.session.post(url, json=ID)
resp.raise_for_status()
ec2 = resp.json()["credential"]

print("EC2 Access:", ec2["access"])
print("EC2 Secret:", ec2["secret"])